In [70]:
import psycopg2
import pandas as pd
import os
import glob
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from datetime import date
from datetime import datetime
import calendar
from openpyxl.utils import get_column_letter
import numpy as np
from sqlalchemy import create_engine


# ACESSO A BASE DE DADOS SQL

In [2]:
conn  = psycopg2.connect(host='localhost',
                           database='CAGED_BRUTO',
                           user='******', 
                           password='******',
                           port ='******')

print(conn .info)
print(conn .status)

1


# IMPORTANDO A BASE

In [3]:
# Query SQL para selecionar os dados da tabela específica no schema 2023 onde a coluna "subclass" começa com "49302"
query_sql = "SELECT * FROM \"2023\".\"11-2023\" WHERE CAST(subclass AS TEXT) LIKE '49302%';"

# Carregar os dados do banco de dados em um DataFrame do pandas
df = pd.read_sql_query(query_sql, conn)

# Fechar a conexão com o banco de dados
conn.close()

# Exibir as primeiras linhas do DataFrame
print(df.head())


C:\Users\ricardo\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   competenciamov  regiao  uf  municipio secao  subclass  saldomovimentacao  \
0          202311       3  33     330170     H   4930202                 -1   
1          202311       4  41     411200     H   4930202                  1   
2          202311       4  41     411520     H   4930202                  1   
3          202311       3  31     317020     H   4930202                  1   
4          202311       2  29     291955     H   4930202                 -1   

   cbo2002ocupacao  categoria  graudeinstrucao  ...  indtrabintermitente  \
0           782510        101                4  ...                    0   
1           642015        101                7  ...                    0   
2           782510        101                7  ...                    0   
3           782510        101                7  ...                    0   
4           215305        101                9  ...                    0   

  indtrabparcial   salario  tamestabjan  indicadoraprendiz  \
0     

In [4]:
# Número de linhas no DataFrame
num_linhas = df.shape[0]

print("Número de linhas no DataFrame:", num_linhas)

Número de linhas no DataFrame: 109418


# TRATAMENTO ETL

In [5]:

# Lista das colunas a serem removidas
colunas_remover = ['secao', 'tipoempregador', 'tipoestabelecimento', 'tipomovimentacao',
                   'tipodedeficiencia', 'indtrabintermitente', 'indtrabparcial',
                   'origemdainformacao', 'competenciadec', 'indicadordeforadoprazo',
                   'unidadesalariocodigo', 'valorsalariofixo']

# Remover as colunas especificadas
df = df.drop(columns=colunas_remover)

# Renomear a coluna 'competenciamov' para 'COD-RELATORIO'
df = df.rename(columns={'competenciamov': 'COD-RELATORIO'})

# Alterar os nomes das colunas para maiúsculo
df.columns = df.columns.str.upper()

In [6]:
df.head()

,COD-RELATORIO,REGIAO,UF,MUNICIPIO,SUBCLASS,SALDOMOVIMENTACAO,CBO2002OCUPACAO,CATEGORIA,GRAUDEINSTRUCAO,IDADE,HORASCONTRATUAIS,RACACOR,SEXO,SALARIO,TAMESTABJAN,INDICADORAPRENDIZ
0,202311,3,33,330170,4930202,-1,782510,101,4,64.0,"44,00",1,1,"2543,25",5,0
1,202311,4,41,411200,4930202,1,642015,101,7,20.0,"44,00",1,1,"2700,00",3,0
2,202311,4,41,411520,4930202,1,782510,101,7,45.0,"44,00",1,1,"2704,23",10,0
3,202311,3,31,317020,4930202,1,782510,101,7,39.0,"44,00",2,1,"2015,88",7,0
4,202311,2,29,291955,4930202,-1,215305,101,9,42.0,"44,00",1,1,"10340,00",2,0


In [72]:
# Fechar o arquivo Excel
workbook_desc_caged_base.close()

In [56]:
# Carregar a planilha de tradução
descricao_caged = r'C:\Users\ricardo\CAGED\DESCRIÇÃO - CAGED.xlsx'

# Carregar o arquivo Excel
workbook_desc_caged_base = load_workbook(filename=descricao_caged, read_only=True)

In [57]:
# Listar todas as abas
abas = workbook_desc_caged_base.sheetnames

# Mostrar as abas
print("Abas presentes no arquivo:")
for aba in abas:
    print(aba)

Abas presentes no arquivo:
Layout
REGIAO
UF
MUNICIPIOS
CBO
categoria
GRAU DE INSTRUCAO
RACA COR
SEXO
FAIXA ETARIA


In [58]:
# Selecionar as abas no arquivo base
aba_regiao = workbook_desc_caged_base['REGIAO']
aba_uf = workbook_desc_caged_base['UF']
aba_municipios = workbook_desc_caged_base['MUNICIPIOS']
aba_cbo = workbook_desc_caged_base['CBO']
aba_categoria = workbook_desc_caged_base['categoria']
aba_grau_de_instrucao = workbook_desc_caged_base['GRAU DE INSTRUCAO']
aba_raca_cor = workbook_desc_caged_base['RACA COR']
aba_sexo = workbook_desc_caged_base['SEXO']
aba_faixa_etaria = workbook_desc_caged_base['FAIXA ETARIA']

# Converter as abas em DataFrame
df_desc_regiao = pd.DataFrame(aba_regiao.values, columns=[cell.value for cell in aba_regiao[1]])
df_desc_uf = pd.DataFrame(aba_uf.values, columns=[cell.value for cell in aba_uf[1]])
df_desc_municipios = pd.DataFrame(aba_municipios.values, columns=[cell.value for cell in aba_municipios[1]])
df_desc_cbo = pd.DataFrame(aba_cbo.values, columns=[cell.value for cell in aba_cbo[1]])
df_desc_categoria = pd.DataFrame(aba_categoria.values, columns=[cell.value for cell in aba_categoria[1]])
df_desc_grau_de_instrucao = pd.DataFrame(aba_grau_de_instrucao.values, columns=[cell.value for cell in aba_grau_de_instrucao[1]])
df_desc_raca_cor = pd.DataFrame(aba_raca_cor.values, columns=[cell.value for cell in aba_raca_cor[1]])
df_desc_sexo = pd.DataFrame(aba_sexo.values, columns=[cell.value for cell in aba_sexo[1]])
df_desc_faixa_etaria = pd.DataFrame(aba_faixa_etaria.values, columns=[cell.value for cell in aba_faixa_etaria[1]])

In [59]:
print("Colunas do DataFrame 'df_desc_regiao':")
print(df_desc_regiao.columns)

print("\nColunas do DataFrame 'df_desc_uf':")
print(df_desc_uf.columns)

print("\nColunas do DataFrame 'df_desc_municipios':")
print(df_desc_municipios.columns)

print("\nColunas do DataFrame 'df_desc_cbo':")
print(df_desc_cbo.columns)

print("\nColunas do DataFrame 'df_desc_categoria':")
print(df_desc_categoria.columns)

print("\nColunas do DataFrame 'df_desc_grau_de_instrucao':")
print(df_desc_grau_de_instrucao.columns)

print("\nColunas do DataFrame 'df_desc_raca_cor':")
print(df_desc_raca_cor.columns)

print("\nColunas do DataFrame 'df_desc_sexo':")
print(df_desc_sexo.columns)

print("\nColunas do DataFrame 'df_desc_faixa_etaria':")
print(df_desc_faixa_etaria.columns)


Colunas do DataFrame 'df_desc_regiao':
Index(['Códigos', 'Descrição'], dtype='object')

Colunas do DataFrame 'df_desc_uf':
Index(['Códigos', 'UFs', 'Descrição'], dtype='object')

Colunas do DataFrame 'df_desc_municipios':
Index(['Códigos', 'Descrição', 'BASE', 'ESTADO'], dtype='object')

Colunas do DataFrame 'df_desc_cbo':
Index(['Códigos', 'Cargo', 'Descrição', 'Área', 'CARGO DO TRC?',
       'TEM RELAÇÃO?', 'Apropriado?', 'Teto salarial 06/2023',
       'Média Salarial'],
      dtype='object')

Colunas do DataFrame 'df_desc_categoria':
Index(['Códigos', 'GRUPO', 'Descrição'], dtype='object')

Colunas do DataFrame 'df_desc_grau_de_instrucao':
Index(['Códigos', 'Descrição'], dtype='object')

Colunas do DataFrame 'df_desc_raca_cor':
Index(['Códigos', 'Descrição'], dtype='object')

Colunas do DataFrame 'df_desc_sexo':
Index(['Códigos', 'Descrição'], dtype='object')

Colunas do DataFrame 'df_desc_faixa_etaria':
Index(['Códigos', 'Descrição'], dtype='object')


In [60]:
#converter df para df_tratado
df_tratado = df.copy()

In [61]:
# Adicionar coluna MÊS ao lado da coluna COD-RELATORIO
df_tratado.insert(df_tratado.columns.get_loc('COD-RELATORIO') + 1, 'MÊS','0')

# Adicionar coluna MÊS ao lado da coluna MUNICIPIO
df_tratado.insert(df_tratado.columns.get_loc('MUNICIPIO') + 1, 'BASE','0')

# Adicionar coluna MÊS ao lado da coluna SALDOMOVIMENTACAO
df_tratado.insert(df_tratado.columns.get_loc('SALDOMOVIMENTACAO') + 1, 'SITUAÇÃO','0')

# Adicionar coluna DESC CBO ao lado da coluna CBO2002OCUPACAO
df_tratado.insert(df_tratado.columns.get_loc('CBO2002OCUPACAO') + 1, 'DESC CBO','0')

# Adicionar coluna MÊS ao lado da coluna DESC CBO
df_tratado.insert(df_tratado.columns.get_loc('DESC CBO') + 1, 'ÁREA','0')

# Adicionar coluna Faixa Etária ao lado da coluna INDICADORAPRENDIZ
df_tratado.insert(df_tratado.columns.get_loc('INDICADORAPRENDIZ') + 1, 'Faixa Etária', '0')

In [62]:
# Dicionário para mapear os últimos dois dígitos do valor da coluna COD-RELATORIO para o mês correspondente
meses = {
    '01': 'JANEIRO',
    '02': 'FEVEREIRO',
    '03': 'MARÇO',
    '04': 'ABRIL',
    '05': 'MAIO',
    '06': 'JUNHO',
    '07': 'JULHO',
    '08': 'AGOSTO',
    '09': 'SETEMBRO',
    '10': 'OUTUBRO',
    '11': 'NOVEMBRO',
    '12': 'DEZEMBRO'
}

# Preencher a coluna MÊS com os nomes dos meses correspondentes aos últimos dois dígitos da coluna COD-RELATORIO
df_tratado['MÊS'] = df_tratado['COD-RELATORIO'].astype(str).str[-2:].map(meses)

# Exibir o DataFrame atualizado
df_tratado.head()

,COD-RELATORIO,MÊS,REGIAO,UF,MUNICIPIO,BASE,SUBCLASS,SALDOMOVIMENTACAO,SITUAÇÃO,CBO2002OCUPACAO,...,CATEGORIA,GRAUDEINSTRUCAO,IDADE,HORASCONTRATUAIS,RACACOR,SEXO,SALARIO,TAMESTABJAN,INDICADORAPRENDIZ,Faixa Etária
0,202311,NOVEMBRO,3,33,330170,0,4930202,-1,0,782510,...,101,4,64.0,"44,00",1,1,"2543,25",5,0,0
1,202311,NOVEMBRO,4,41,411200,0,4930202,1,0,642015,...,101,7,20.0,"44,00",1,1,"2700,00",3,0,0
2,202311,NOVEMBRO,4,41,411520,0,4930202,1,0,782510,...,101,7,45.0,"44,00",1,1,"2704,23",10,0,0
3,202311,NOVEMBRO,3,31,317020,0,4930202,1,0,782510,...,101,7,39.0,"44,00",2,1,"2015,88",7,0,0
4,202311,NOVEMBRO,2,29,291955,0,4930202,-1,0,215305,...,101,9,42.0,"44,00",1,1,"10340,00",2,0,0


In [63]:
df_tratado['REGIAO'] = df_tratado['REGIAO'].map(df_desc_regiao.set_index('Códigos')['Descrição'])
df_tratado['UF'] = df_tratado['UF'].map(df_desc_uf.set_index('Códigos')['Descrição'])
df_tratado['MUNICIPIO'] = df_tratado['MUNICIPIO'].map(df_desc_municipios.set_index('Códigos')['Descrição'])
df_tratado['BASE'] = df_tratado['MUNICIPIO'].map(df_desc_municipios.set_index('Descrição')['BASE'])
df_tratado['DESC CBO'] = df_tratado['CBO2002OCUPACAO'].map(df_desc_cbo.set_index('Códigos')['Descrição'])
df_tratado['ÁREA'] = df_tratado['CBO2002OCUPACAO'].map(df_desc_cbo.set_index('Códigos')['Área'])
df_tratado['CATEGORIA'] = df_tratado['CATEGORIA'].map(df_desc_categoria.set_index('Códigos')['Descrição'])
df_tratado['GRAUDEINSTRUCAO'] = df_tratado['GRAUDEINSTRUCAO'].map(df_desc_grau_de_instrucao.set_index('Códigos')['Descrição'])
df_tratado['RACACOR'] = df_tratado['RACACOR'].map(df_desc_raca_cor.set_index('Códigos')['Descrição'])
df_tratado['SEXO'] = df_tratado['SEXO'].map(df_desc_sexo.set_index('Códigos')['Descrição'])
df_tratado['Faixa Etária'] = df_tratado['IDADE'].map(df_desc_faixa_etaria.set_index('Códigos')['Descrição'])

In [66]:
# Substituir os valores na coluna SITUAÇÃO com base na coluna SALDOMOVIMENTACAO
df_tratado['SITUAÇÃO'] = np.where(df_tratado['SALDOMOVIMENTACAO'] == 1, 'ADMITIDO', 'DEMITIDO')

In [67]:
df_tratado.head()

,COD-RELATORIO,MÊS,REGIAO,UF,MUNICIPIO,BASE,SUBCLASS,SALDOMOVIMENTACAO,SITUAÇÃO,CBO2002OCUPACAO,...,CATEGORIA,GRAUDEINSTRUCAO,IDADE,HORASCONTRATUAIS,RACACOR,SEXO,SALARIO,TAMESTABJAN,INDICADORAPRENDIZ,Faixa Etária
0,202311,NOVEMBRO,SUDESTE,RIO DE JANEIRO,DUQUE DE CAXIAS,GERAL,4930202,-1,DEMITIDO,782510,...,"Empregado - Geral, inclusive o empregado públi...",5ª a 8ª série incompleta do 1º grau (ensino fu...,64.0,"44,00",Branca,HOMEM,"2543,25",5,0,Mais de 60 anos
1,202311,NOVEMBRO,SUL,PARANA,JAGUARIAIVA,GERAL,4930202,1,ADMITIDO,642015,...,"Empregado - Geral, inclusive o empregado públi...",2º grau completo (ensino médio),20.0,"44,00",Branca,HOMEM,"2700,00",3,0,menos de 20 anos
2,202311,NOVEMBRO,SUL,PARANA,MARINGA,GERAL,4930202,1,ADMITIDO,782510,...,"Empregado - Geral, inclusive o empregado públi...",2º grau completo (ensino médio),45.0,"44,00",Branca,HOMEM,"2704,23",10,0,41-45
3,202311,NOVEMBRO,SUDESTE,MINAS GERAIS,UBERLANDIA,GERAL,4930202,1,ADMITIDO,782510,...,"Empregado - Geral, inclusive o empregado públi...",2º grau completo (ensino médio),39.0,"44,00",Preta,HOMEM,"2015,88",7,0,36-40
4,202311,NOVEMBRO,NORDESTE,BAHIA,LUIS EDUARDO MAGALHAES,GERAL,4930202,-1,DEMITIDO,215305,...,"Empregado - Geral, inclusive o empregado públi...",Superior Completo,42.0,"44,00",Branca,HOMEM,"10340,00",2,0,41-45


In [69]:
# Tipos de dados das colunas no DataFrame tratado
tipos_de_dados_df = df_tratado.dtypes

# Imprimir os tipos de dados das colunas
print(tipos_de_dados_df)


COD-RELATORIO          int64
MÊS                   object
REGIAO                object
UF                    object
MUNICIPIO             object
BASE                  object
SUBCLASS               int64
SALDOMOVIMENTACAO      int64
SITUAÇÃO              object
CBO2002OCUPACAO        int64
DESC CBO              object
ÁREA                  object
CATEGORIA             object
GRAUDEINSTRUCAO       object
IDADE                float64
HORASCONTRATUAIS      object
RACACOR               object
SEXO                  object
SALARIO               object
TAMESTABJAN            int64
INDICADORAPRENDIZ      int64
Faixa Etária          object
dtype: object


# SALVAR ARQUIVO EM XLSX

In [68]:
# Definir o nome do arquivo com base no mês e ano
nome_arquivo = "CAGED_" + str(df_tratado['COD-RELATORIO'].iloc[0]) + ".xlsx"

# Diretório local para salvar o arquivo baixado
local_dir = "C:/Users/ricardo/Downloads"

# Caminho completo do arquivo
caminho_arquivo = os.path.join(local_dir, nome_arquivo)

# Salvar o DataFrame como um arquivo Excel no diretório especificado
df_tratado.to_excel(caminho_arquivo, index=False)

# Exibir mensagem de confirmação
print("Arquivo salvo como:", nome_arquivo)


Arquivo salvo como: CAGED_202311.xlsx


# SALVAR O ARQUIVO NO BANCO DE DADOS

In [71]:
# Criar uma conexão com o banco de dados PostgreSQL
conn = psycopg2.connect(
    host="******",
    database="******",
    user="******",
    password="******"
)

# Criar um mecanismo SQLAlchemy para se conectar ao banco de dados
engine = create_engine('postgresql://******:******@******/CAGED_TRATADO')

# Salvar o DataFrame em uma tabela específica no banco de dados
schema_name = 'ano_2023'
table_name = '11-2023'
df_tratado.to_sql(table_name, engine, schema=schema_name, if_exists='replace', index=False)

# Fechar a conexão com o banco de dados
conn.close()